In [2]:
import pandas as pd
import numpy as np
import duckdb


In [3]:
# print(os.getcwd())

In [4]:
# dimension table
product_master = pd.read_parquet('../dataset/parquet/PRODUCT_MASTER.parquet')
store_master = pd.read_parquet('../dataset/parquet/STORE_MASTER.parquet')
user_master = pd.read_parquet('../dataset/parquet/USER_MASTER.parquet')
# fact tables
operation_details = pd.read_parquet('../dataset/parquet/FACT_OPERATION_TABLE.parquet')
order_details = pd.read_parquet('../dataset/parquet/FACT_ORDER_TABLE.parquet')
# RFM table
rfm_table = pd.read_parquet('../csv_export/RFM.parquet')

In [9]:
# convert columns to datetime amd time
order_details['ORDER_TIMESTAMP'] = pd.to_datetime(order_details['ORDER_TIMESTAMP'], errors='coerce')
order_details['ORDER_DATE'] = pd.to_datetime(order_details['ORDER_DATE'], errors='coerce')
order_details['ORDER_TIME'] = pd.to_datetime(order_details['ORDER_TIME'], format='%H:%M:%S', errors='coerce').dt.time

In [11]:
# connection to DuckDB in-memory database
con = duckdb.connect(database=':memory:')
con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='2GB'")

In [12]:
# load tables
con.register('product_master', product_master)
con.register('store_master', store_master)
con.register('user_master', user_master)
con.register('operation_details', operation_details)
con.register('order_details', order_details)

In [13]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2060463 entries, 0 to 2060462
Data columns (total 31 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   ORDER_DATE                    datetime64[ns]
 1   ORDER_TIME                    object        
 2   ORDER_TIMESTAMP               datetime64[ns]
 3   UNIT_QUANTITY                 int64         
 4   UNIT_VOLUME                   float64       
 5   UNIT_LIST_PRICE               float64       
 6   UNIT_OFFERED_PRICE            float64       
 7   UNIT_CHARGED_PRICE            float64       
 8   UNIT_DELIVERY_PRICE           float64       
 9   UNIT_DISCOUNT                 float64       
 10  UNIT_TAX                      float64       
 11  UNIT_GROSS                    float64       
 12  UNIT_NET                      float64       
 13  ORDER_STATE                   object        
 14  ORDER_STATE_FINAL             object        
 15  CUSTOMER_FIRST_ORDER_PERIOD   in

In [21]:
duckdb.query("""
SELECT YEAR(ORDER_DATE) AS ORDER_YEAR
    , COUNT(DISTINCT CUSTOMER_ID) AS UNIQUE_CUSTOMERS
    , COUNT(DISTINCT DEVICE_ID) AS UNIQUE_DEVICES
FROM order_details
WHERE TRUE
AND ORDER_STATE_FINAL = 'Valid'
AND YEAR(ORDER_DATE) >= 2022
GROUP BY 1
ORDER BY 1
""").df()

,ORDER_YEAR,UNIQUE_CUSTOMERS,UNIQUE_DEVICES
0,2022,58481,38628
1,2023,84807,75487
2,2024,106650,123188
3,2025,58905,60570


In [19]:
duckdb.query("""
SELECT YEAR(ORDER_DATE) AS ORDER_YEAR
    , MONTH(ORDER_DATE) AS ORDER_MONTH
    , COUNT(DISTINCT CUSTOMER_ID) AS UNIQUE_CUSTOMERS
    , COUNT(DISTINCT DEVICE_ID) AS UNIQUE_DEVICES
FROM order_details
WHERE TRUE
AND ORDER_STATE_FINAL = 'Valid'
AND YEAR(ORDER_DATE) >= 2024
GROUP BY 1, 2
ORDER BY 1, 2
""").df()

,ORDER_YEAR,ORDER_MONTH,UNIQUE_CUSTOMERS,UNIQUE_DEVICES
0,2024,1,14098,14092
1,2024,2,18536,18598
2,2024,3,20012,20146
3,2024,4,17881,17990
4,2024,5,17841,17898
5,2024,6,20791,21305
6,2024,7,19293,19330
7,2024,8,21035,21031
8,2024,9,19993,19758
9,2024,10,24044,23928


In [20]:
duckdb.query("""
SELECT YEAR(ORDER_DATE) AS ORDER_YEAR
    , MONTH(ORDER_DATE) AS ORDER_MONTH
    , COUNT(DISTINCT CUSTOMER_ID) AS UNIQUE_CUSTOMERS
    , COUNT(DISTINCT DEVICE_ID) AS UNIQUE_DEVICES
FROM order_details
WHERE TRUE
AND ORDER_STATE_FINAL = 'Valid'
AND YEAR(ORDER_DATE) >= 2022
GROUP BY 1, 2
ORDER BY 1, 2
""").df()

,ORDER_YEAR,ORDER_MONTH,UNIQUE_CUSTOMERS,UNIQUE_DEVICES
0,2022,1,1924,588
1,2022,2,2366,1213
2,2022,3,3074,2188
3,2022,4,3362,2447
4,2022,5,3022,2056
5,2022,6,4499,3295
6,2022,7,6145,5051
7,2022,8,7873,6069
8,2022,9,11338,8893
9,2022,10,11898,9594


In [ ]:
# duckdb.query("""
# SELECT ORDER_ID
#         , ORDER_TIMESTAMP
#         , CUSTOMER_ID
# FROM order_details
# WHERE TRUE
# AND ORDER_DATE >= '2024-01-01'
# ORDER BY ORDER_DATE DESC
# """).df()

In [ ]:
duckdb.query("""
SELECT *
FROM order_details
WHERE TRUE
AND ORDER_DATE >= '2025-01-01'   
ORDER BY ORDER_TIMESTAMP DESC
""").df()

,ORDER_DATE,ORDER_TIME,ORDER_TIMESTAMP,UNIT_QUANTITY,UNIT_VOLUME,UNIT_LIST_PRICE,UNIT_OFFERED_PRICE,UNIT_CHARGED_PRICE,UNIT_DELIVERY_PRICE,UNIT_DISCOUNT,...,TOTAL_VOLUME,SKU_ID,ORDER_ID,CUSTOMER_ID,COUPON_ID,DISCUNT_TYPE_ID,OPERATOR_ID,CAMPAIGN_ID,DEVICE_ID,STORE_ID
0,2025-03-31,23:00:00,2025-03-31 23:59:12.090,1,0.03780,39.00,36.50,36.50,4.99,0.0,...,0.03780,75fdfe48e364de97565fe0ed4242fba21bc2f2bfcb638b...,60a16512930c164515abfc6dbb75b4ce2c5d4de9a0c152...,bcbc871d8d9f3eaf1951b580195ca69dc621681b136b19...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,50d177afe701365facbb8a541784d166b9392e458ad53d...,62a36e9503e93e23e84674c55b755826b3d6c5ef0a5082...,a94d2b967fcf164d19462f1aa556fe10de2b4d16571196...,50d177afe701365facbb8a541784d166b9392e458ad53d...
1,2025-03-31,23:00:00,2025-03-31 23:58:43.443,1,0.00750,69.90,69.90,69.90,4.99,0.0,...,0.00750,21717715e896103af2b9bf8d875e4daa60108eeff5be0c...,7dec6bbe45a5101fd0e790d886d967de7c2763a6ed40f5...,65129edf66aeb8d2118e452ff95db23f49474349f22494...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,a68666629f1dbcd87a1bacc0d1d8410ca4c6ee6edf431a...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,6155bdcbc530acd8f5e4c448cfb544e65ee1852d38aa85...,a68666629f1dbcd87a1bacc0d1d8410ca4c6ee6edf431a...
2,2025-03-31,23:00:00,2025-03-31 23:58:33.515,1,0.15120,156.00,139.90,139.90,4.99,0.0,...,0.15120,d4f500c341b585d8d20f6a2ef5277e664cd2bbf147dea0...,e8ccf9483837978c5cced4e07dd01446c04f4e397aa7e0...,92f460ae95c77bcd1b739a4333ab4dd68e7e8ea1e9347d...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,a68666629f1dbcd87a1bacc0d1d8410ca4c6ee6edf431a...,4a8d6b1cd5b22df637dad282d34de24e10697e54333ccd...,69a2d5112e14192b073e2f6a6e4e81b90a618f70576043...,a68666629f1dbcd87a1bacc0d1d8410ca4c6ee6edf431a...
3,2025-03-31,23:00:00,2025-03-31 23:57:07.622,1,0.02000,19.69,19.69,19.69,2.42,0.0,...,0.02000,c108cd05b1b5060f2d8d1c9fc85e395cee424ebdc1847c...,ad973723dc292ee69530ae0429a3ffb20fa03866b74e37...,d52cf4c0a3e99da0fc6f6919f2792657d03a456f6e1464...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,8af66ff75b7154be5590b2a8883ad564afa5be075d2ffa...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,0b83635dff4e6e5c3b77825f3d07ae0dee7d432f641c35...,8af66ff75b7154be5590b2a8883ad564afa5be075d2ffa...
4,2025-03-31,23:00:00,2025-03-31 23:57:07.622,1,0.02838,28.00,20.90,20.90,2.57,0.0,...,0.02838,6e358b6a89089bae28e6b3bcdf3ca9483caf4ff10e4135...,ad973723dc292ee69530ae0429a3ffb20fa03866b74e37...,d52cf4c0a3e99da0fc6f6919f2792657d03a456f6e1464...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,8af66ff75b7154be5590b2a8883ad564afa5be075d2ffa...,9c2acf9bda58ffec8e627696f1adaecb06b995e7858231...,0b83635dff4e6e5c3b77825f3d07ae0dee7d432f641c35...,8af66ff75b7154be5590b2a8883ad564afa5be075d2ffa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285447,2025-01-01,00:00:00,2025-01-01 00:06:25.492,2,0.07560,78.00,69.90,69.90,4.90,0.0,...,0.15120,9f581332f72dfcfda5db2a872374ba7f86dc5d5cb8fc74...,febc66b943730f470b7017db5e8c52c3e59035827aae5b...,6a28ee6a5f06c58f051dc4af68aa06318a2379713c7754...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,e79ad689dac1fef48b1214f2d03867cf0853ba24989838...,b537b60b3c5f326fd603338b759b0e1b1e7b0eecba606d...,91b9d2fe89af1b8b692e2997cd272357bc3d6cae20f919...,e79ad689dac1fef48b1214f2d03867cf0853ba24989838...
285448,2025-01-01,00:00:00,2025-01-01 00:03:49.922,2,0.05676,60.00,60.00,60.00,4.90,0.0,...,0.11352,c2a380a9a2ef5d927c663b2fb35cdfa1e465bb0d679aaf...,7ad3514c85229262e0957138e01e03d96e0de309cb7144...,315c3749a7ca258e8b8aa4dc66fabad9ef6bf5ba44a1f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,e79ad689dac1fef48

In [ ]:
duckdb.query("""
SELECT *
FROM operation_details
WHERE TRUE
AND ORDER_DATE >= '2025-01-01'   
ORDER BY ORDER_DATE, ORDER_TIME DESC
""").df()

,ORDER_DATE,ORDER_TIME,ORDER_STATE,ORDER_STATE_FINAL,PAYMENT_TIMESTAMP,COMPLETION_DATE,COMPLETION_TIME,DELIVERY_TIME,APPROVAL_TIME,WITHIN_SLA,...,VOLUME,START_TIME,STARTED_DISTANCE,TOTAL_DISTANCE,DISTANCE_STORE_FROM_CUSTOMER,IS_FIRST_PURCHASE,ORDER_ID,OPERATOR_ID,FLEET_ID,STORE_ID
0,2025-01-01,23:59:42,Valid,Valid,None,2025-01-02,00:30:06,30.0,0.0,1,...,0.03228,13.0,NaN,0.068769,3.957220,False,07ab9e683163e25966385650f0d3154bfefe3b2d6d42f9...,860b5b6ef21d3354d7da049f87dcfe52a105b0ff7fdb93...,0d06b8f9958d9e237fa3fb899dc284d4b428a1d32a4caa...,860b5b6ef21d3354d7da049f87dcfe52a105b0ff7fdb93...
1,2025-01-01,23:59:21,Valid,Valid,None,2025-01-02,00:20:43,21.0,0.0,1,...,0.04260,12.0,NaN,0.015251,3.356034,False,17b079e934e30e9e0e63bc1e1ba06b14a32ece359a1a63...,a68666629f1dbcd87a1bacc0d1d8410ca4c6ee6edf431a...,272b88d48251a90abba53a1a06e029d233f4483a734866...,a68666629f1dbcd87a1bacc0d1d8410ca4c6ee6edf431a...
2,2025-01-01,23:58:27,Valid,Valid,None,2025-01-02,00:32:03,33.0,1.0,1,...,0.02838,3.0,NaN,0.026458,2.005231,False,a96ac5777fa9ff14b31c9e3f285380a0cb1837bb111ece...,940d5c8a453555cb2101f8fd823321f17f0af66d820c9e...,2a5d59b92aea82237e5490e62becdecbca8b3d72884ace...,940d5c8a453555cb2101f8fd823321f17f0af66d820c9e...
3,2025-01-01,23:57:11,Valid,Valid,None,2025-01-02,00:30:21,33.0,0.0,1,...,0.07560,8.0,NaN,0.080807,1.191647,True,833d998e83901113bfa8f5d3ed90ff4bbb70855bd4392a...,04303dddd737ef9dc0a6cfab01dae96087d887c7855476...,6a7828955b43cc0df325b4ffc237fc01b7c7c5b4d3d6f8...,04303dddd737ef9dc0a6cfab01dae96087d887c7855476...
4,2025-01-01,23:57:07,Valid,Valid,None,2025-01-02,00:22:02,24.0,0.0,1,...,0.01419,4.0,NaN,0.023329,1.138686,False,4b3b54056292384d5f1b2836c3ac42bb5c7bcd5c97c2dc...,940d5c8a453555cb2101f8fd823321f17f0af66d820c9e...,2a5d59b92aea82237e5490e62becdecbca8b3d72884ace...,940d5c8a453555cb2101f8fd823321f17f0af66d820c9e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219048,2025-03-31,00:03:52,Valid,Valid,None,2025-03-31,00:23:17,19.0,0.0,1,...,0.02050,11.0,NaN,0.005907,1.912512,False,f1bd2804205251a2e1b278c38fb69efc82ecb35ccab6ff...,9dbdfdf20ce5ab69fe18b4c8b6377b319b9ba5021b0809...,699d65492a9ff4b14e07f0526fbfe30f427e24fecdba7d...,9dbdfdf20ce5ab69fe18b4c8b6377b319b9ba5021b0809...
219049,2025-03-31,00:01:42,Invalid,Invalid,None,2025-03-31,00:02:00,0.0,0.0,1,...,0.03780,7.0,NaN,NaN,4.207513,False,9126c17948d7e9e3dc51f55880b6a62035b4ac34317906...,b5dc19990b33d6005bc1d81e8cc1bf19f6d1ed7bb59009...,6eb199f1cb66c75c3b4ed2731cc917cfddcc8f14573137...,b5dc19990b33d6005bc1d81e8cc1bf19f6d1ed7bb59009...
219050,2025-03-31,00:01:42,Invalid,Invalid,None,2025-03-31,00:02:26,0.0,0.0,1,...,0.00700,0.0,NaN,NaN,2.028624,False,5e163e313dde6d38fafc178bc13cc5e553efb7eb4f55ec...,25cb2e263c6122bbf3520c89acb32ed591b9660a4efb60...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,25cb2e263c6122bbf3520c89acb32ed591b9660a4efb60...
219051,2025-03-31,00:00:29,Valid,Valid,None,2025-03-31,00:21:11,20.0,0.0,1,...,0.07320,1.0,NaN,1.005645,0.981735,True,dafa4ade239b6d3966177907a05b83dfebab4cf363258b...,7b9a4e15d45ea59cd62b322f848e37fb2c0db1fbd05e83...,d5fd7ba15d244a0347b35e30a0ae8ef8277f7fe5fe3432...,7b9a4e15d45ea59cd62b322f848e37fb2c0db1fbd05e83...


In [ ]:
duckdb.query("""
SELECT *
FROM product_master
""").df()

,SKU_ID,SKU_NAME,SKU_BRAND,SKU_CATEGORY,SKU_SUBCATEGORY,SKU_RETURNABLE_CONTAINER,SKU_ALCOHOLIC,SKU_OWN_CATALOG,SKU_CONTAINER,SKU_IS_RETURNABLE,SKU_IS_BUNDLE
0,fbae0786652f83ddc651d0cd36188b40103f33d9b5ff7f...,(2) Beer Budweiser Bottle 343 ml 6 un,Budweiser,Beer,Beer,None,1,1,NRB,false,0
1,b33722a436bb87d876e490d347a15c32cc6a61b61725e4...,Paulaner Weiss Can (269ml) Pack x 6,Paulaner,Beer,Beer,None,1,1,CAN,0,0
2,9c73ed825c15f3587dbaae07e4f01c8fb152cf2d5f0d2c...,(1) Beer Pilsator Can 355 ml 6 un,Pilsator,Beer,Beer,None,1,1,CAN,0,0
3,8c76e6477fec2c1b69a8c5026b885352da383f1d3dc637...,(1) Beer San Miguel Can 355 ml 6 un,San Miguel,Beer,Beer,None,1,1,CAN,false,0
4,8084efed817688b3b61a3a9ee104e0d08476e6b9b0fdf9...,(1) Energy drink Red Bull Can 250 ml 1 un,Red Bull,Drinks,Energy Drink,None,0,0,-,false,0
...,...,...,...,...,...,...,...,...,...,...,...
1544,2d0840703c98c227f79a3c258949896ac5f076d99e39bc...,2 Carlsberg BEERCAKE Can 355 ML Twelvepack 1 C...,Carlsberg,Beer,Beer,None,1,1,CAN,0,0
1545,2026415734d07f1735218b3aa97843262d4b4dccaa89dd...,(3) Paulaner Hell 310ML Non Returnable 6X1,Paulaner,Beer,Beer,0,1,1,NRB,0,0
1546,5bfc19eedc75d2b2514284917c295621549c32599380ae...,(1) Shirt Carlsberg Blanco Anniversary,Carlsberg,Beer,Beer,None,1,1,-,false,0
1547,d2b61df541a7b3631bb7d58967dc2326335344dd04ef5f...,Paulaner Hell Bottle 310ml Sixpack Corona Bott...,Paulaner,Beer,Beer,None,1,1,NRB,0,0


In [ ]:
duckdb.query("""
SELECT *
FROM store_master
""").df()

,STORE_ID,STORE_NAME,STATUS,CITY,PREFIX,DENOMINATION
0,b4dbd85fdc14f93f1db4a80538a5d3ee3184d606e6d00e...,Nairobi,ACTIVE,MIA,DS,darkstore
1,3a9280a7ef0535508e92e5bbc743b6af83c358a176377d...,Belmopan,ACTIVE,MIA,DS,darkstore
2,2df16f89e221264d08396b08e45fd3a8cd71ececdf947c...,Andorra la Vella,ACTIVE,MIA,DS,darkstore
3,bc853a9baae81a14cc2a279c23f7b9af149588f80b7ffd...,Madrid,ACTIVE,MIA,DS,darkstore
4,af32eedaaea7c5c9cb8fa60c08d1c2a1d878104a054698...,San Jose,ACTIVE,MIA,DS,darkstore
...,...,...,...,...,...,...
79,aed846e188d50afb8e221a8eda2adc58fd897e1b9a353f...,Sofia,ACTIVE,MIA,WH,warehouse
80,8ebacabc393f66d5c3ba210dca34d542bc4c94b93ccd53...,Vienna,ACTIVE,MIA,WH,warehouse
81,ad969f1ced7cfe52860069d4b6ee35d89e08b67db7a1e1...,Accra,ACTIVE,MIA,WH,warehouse
82,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...,Ottawa,ACTIVE,MIA,ST,store


In [ ]:
rfm_df = duckdb.query("""
SELECT *
FROM rfm_table
""").df()

In [ ]:
rfm_table

,CUSTOMER_ID,Recency,Frequency,Total_Spent,Total_Units,Total_Organic_Discount,Total_Extra_Discount,Total_Discounts,Monetary,AVG_ORGANIC_DISCOUNT_PER_ORDER,...,CATEGORY_DIVERSITY,FAVORITE_CATEGORY,SUB_CATEGORY_DIVERSITY,FAVORITE_SUB_CATEGORY,AVG_DELIVERY_TIME,SLA_VIOLATION_RATE,AVG_APPROVAL_TIME,AVG_TOTAL_DISTANCE,FAVORITE_WEEKDAY,FAVORITE_HOUR
0,00002b303bd5e3641c7deb26f7a66bd83b202c4da03b08...,487,1,61.3900,1,19.11,0.0000,19.1100,61.3900,19.110000,...,1,Beer,1,Beer,31.000000,0.000000,0.000000,0.008983,Thursday,12
1,00008655ece97d61c76ec17e483ac6fd7813e5a15c498a...,3,5,670.8799,6,86.53,41.7701,128.3001,670.8799,17.306000,...,2,Beer,2,Beer,29.800000,0.000000,0.000000,0.064991,Monday,11
2,0000bc0f1d0509b07a0472ce0be544d25e1d90a83dcf3a...,109,7,610.5100,16,2.59,107.0000,109.5900,610.5100,0.370000,...,2,Beer,2,Beer,23.000000,0.000000,1.285714,0.045375,Saturday,13
3,0000d61a21fef1a1a0be35e5ad84acaad0af9039d7096a...,491,1,30.3882,1,4.00,0.0118,4.0118,30.3882,4.000000,...,1,Beer,1,Beer,9.000000,0.000000,0.000000,0.007809,Sunday,13
4,00013de18abb79cce6126077cdd1911d5e02141dc1b5d9...,44,27,2187.3447,55,320.90,85.1053,406.0053,2187.3447,11.885185,...,5,Beer,7,Beer,21.555556,0.037037,1.074074,0.121917,Saturday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223388,fffe6449bfcbca35a9f7612e7e9916835087643d014ab5...,148,1,52.7000,1,14.10,0.0000,14.1000,52.7000,14.100000,...,1,Beer,1,Beer,21.000000,0.000000,0.000000,0.026411,Sunday,21
223389,fffea8b48622d5d963ced49f8050e80d5ce8d73253ca3b...,32,23,1409.2700,66,60.70,67.4000,128.1000,1409.2700,2.639130,...,5,Cigarrete,6,Cigarrete,24.608696,0.043478,0.217391,0.574881,Sunday,18
223390,fffeb2919b9035be32bb443948672cb3ff6fd392ae8427...,2,8,544.2700,8,78.82,0.0000,78.8200,544.2700,9.852500,...,1,Beer,1,Beer,29.375000,0.000000,0.000000,0.309222,Saturday,2
223391,ffff67b4e149169036c25b85b78b2212f6337791625877...,582,13,1323.6000,28,0.00,327.6000,327.6000,1323.6000,0.000000,...,2,Beer,2,Beer,17.000000,0.076923,1.000000,0.035138,Friday,0
